In [22]:
import snowflake.connector as snwconn
from snowflake.connector.pandas_tools import write_pandas

import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

True

In [23]:
conn_params = {
    'account': os.getenv('SNW_ACCOUNT').replace('.', '-'),
    'user': os.getenv('SNW_USERNAME'),
    'password': os.getenv('SNW_PASS'),
    'warehouse': 'COMPUTE_WH',
    'database': 'JMAN_FINAL_PROJECT',
    'schema': 'DBO'
}

In [24]:
conn = snwconn.connect(**conn_params)
cursor = conn.cursor()

In [25]:
def _upload_table(table_name, df):

    df.columns = map(lambda x: str(x).upper(), df.columns)
    df_schema = pd.io.sql.get_schema(df, table_name)
    df_schema = str(df_schema).replace('CREATE', 'CREATE OR REPLACE').replace(' TEXT', ' VARCHAR(5000)').replace(' REAL', ' VARCHAR(5000)')

    # print('Executing: \n' + df_schema)
    cursor.execute(df_schema)

    success, _, nrows, _ = write_pandas(conn, df, table_name)
    print(f"table: {table_name:25s} | status: {str(success):5s} | rows: {nrows:5d}\n")

In [26]:
users_df = pd.read_csv('csvs/users.csv')
project_df = pd.read_csv('csvs/projects.csv')
feedbackQuestion_df = pd.read_csv('csvs/feedbackQuestions.csv')
projectsToUsers_df = pd.read_csv('csvs/projectsToUsers.csv')
projectsToFeedbackQuestions_df = pd.read_csv('csvs/projectsToFeedbackQuestions.csv')
timesheets_df = pd.read_csv('csvs/timesheets.csv')
feedbackAnswers_df = pd.read_csv('csvs/feedbackAnswers.csv')

In [27]:
_upload_table('USERS', users_df)
_upload_table('PROJECTS', project_df)
_upload_table('FEEDBACKQUESTIONS', feedbackQuestion_df)
_upload_table('PROJECTSTOUSERS', projectsToUsers_df)
_upload_table('PROJECTSTOFEEDBACKQUESTIONS', projectsToFeedbackQuestions_df)
_upload_table('TIMESHEETS', timesheets_df)
_upload_table('FEEDBACKANSWERS', feedbackAnswers_df)

table: USERS                     | status: True  | rows:  5000

table: PROJECTS                  | status: True  | rows:    50

table: FEEDBACKQUESTIONS         | status: True  | rows:    20

table: PROJECTSTOUSERS           | status: True  | rows: 127278

table: PROJECTSTOFEEDBACKQUESTIONS | status: True  | rows:   521

table: TIMESHEETS                | status: True  | rows: 170858

table: FEEDBACKANSWERS           | status: True  | rows:  8000



In [28]:
cursor.close()
conn.close()